# Crowdsourcing Tutorial

In this tutorial, we'll provide a simple walkthrough of how to use Snorkel in conjunction with crowdsourcing to create a training set for a sentiment analysis task.
We already have crowdsourced labels for about half of the training dataset.
The crowdsourced labels are fairly accurate, but do not cover the entire training dataset, nor are they available for the test set or during inference.
To make up for their lack of training set coverage, we combine crowdsourced labels with heuristic labeling functions to increase the number of training labels we have.
Like most Snorkel labeling pipelines, we'll use the denoised labels to train a deep learning
model which can be applied to new, unseen data to automatically make predictions.

## Dataset Details

In this tutorial, we'll use the [Weather Sentiment](https://data.world/crowdflower/weather-sentiment) dataset from Figure Eight.
Our goal is to train a classifier that can label new tweets as expressing either a positive or negative sentiment.

Crowdworkers were asked to label the sentiment of a particular tweet relating to the weather.
The catch is that 20 crowdworkers graded each tweet, and in many cases crowdworkers assigned conflicting sentiment labels to the same tweet.
This is a common issue when dealing with crowdsourced labeling workloads.

Label options were positive, negative, or one of three other options saying they weren't sure if it was positive or negative; we use only the positive/negative labels.
We've also altered the dataset to reflect a realistic crowdsourcing pipeline where only a subset of our available training set has received crowd labels.

We will treat each crowdworker's labels as coming from a single labeling function (LF).
This will allow us to learn a weight for how much to trust the labels from each crowdworker.
We will also write a few heuristic labeling functions to cover the data points without crowd labels.
Snorkel's ability to build high-quality datasets from multiple noisy labeling signals makes it an ideal framework to approach this problem.

## Loading Crowdsourcing Dataset

We start by loading our data which has 287 data points in total.
We take 50 for our development set and 50 for our test set.
The remaining 187 data points form our training set.
Since the dataset is already small, we skip using a validation set.
Note that this very small dataset is primarily used for demonstration purposes here.
In a real setting, we would expect to have access to many more unlabeled tweets, which could help us to train a higher quality model.

In [1]:
import os

# Make sure we're in the right directory
if os.path.basename(os.getcwd()) == "snorkel-tutorials":
    os.chdir("crowdsourcing")

In [2]:
from data import load_data

crowd_labels, df_train, df_dev, df_test = load_data()
Y_dev = df_dev.sentiment.values
Y_test = df_test.sentiment.values

First, let's take a look at our development set to get a sense of what the tweets look like.
We use the following label convention: 0 = Negative, 1 = Positive.

In [3]:
import pandas as pd

# Don't truncate text fields in the display
pd.set_option("display.max_colwidth", 0)

df_dev.head()

,tweet_id,tweet_text,sentiment
tweet_id,,,
79197834,79197834,@mention not in sunny dover! haha,1
80059939,80059939,It is literally pissing it down in sideways rain. I have nothing to protect me from this monstrous weather.,0
79196441,79196441,"Dear perfect weather, thanks for the vest lunch hour of all time. (@ Lady Bird Lake Trail w/ 2 others) {link}",1
84047300,84047300,RT @mention: I can't wait for the storm tonight :),1
83255121,83255121,60 degrees. And its almost the end of may. Wisconsin... I hate you.,0


Now let's take a look at the crowd labels.
We'll convert these into labeling functions.

In [4]:
crowd_labels.head()

,worker_id,label
tweet_id,,
82510997,18034918,1
82510997,7450342,1
82510997,18465660,1
82510997,17475684,0
82510997,14472526,1


## Writing Labeling Functions
Each crowdworker can be thought of as a single labeling function,
as each worker labels a subset of data points,
and may have errors or conflicting labels with other workers / labeling functions.
So we create one labeling function per worker.
We'll simply return the label the worker submitted for a given tweet, and abstain
if they didn't submit a label for it.

### Crowdworker labeling functions

In [5]:
labels_by_annotator = crowd_labels.groupby("worker_id")
worker_dicts = {}
for worker_id in labels_by_annotator.groups:
    worker_df = labels_by_annotator.get_group(worker_id)[["label"]]
    worker_dicts[worker_id] = dict(zip(worker_df.index, worker_df.label))

print("Number of workers:", len(worker_dicts))

Number of workers: 100


In [6]:
from snorkel.labeling import LabelingFunction

ABSTAIN = -1


def worker_lf(x, worker_dict):
    return worker_dict.get(x.tweet_id, ABSTAIN)


def make_worker_lf(worker_id):
    worker_dict = worker_dicts[worker_id]
    name = f"worker_{worker_id}"
    return LabelingFunction(name, f=worker_lf, resources={"worker_dict": worker_dict})


worker_lfs = [make_worker_lf(worker_id) for worker_id in worker_dicts]

Let's take a quick look at how well they do on the development set.

In [7]:
from snorkel.labeling import PandasLFApplier

applier = PandasLFApplier(worker_lfs)
L_train = applier.apply(df_train)
L_dev = applier.apply(df_dev)

  0%|          | 0/187 [00:00<?, ?it/s]

 56%|█████▌    | 104/187 [00:00<00:00, 1031.43it/s]

100%|██████████| 187/187 [00:00<00:00, 1031.16it/s]


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:00<00:00, 1021.63it/s]

Note that because our dev set is so small and our LFs are relatively sparse, many LFs will appear to have zero coverage.
Fortunately, our label model learns weights for LFs based on their outputs on the training set, which is generally much larger.

In [8]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L_dev, worker_lfs).lf_summary(Y_dev).sample(5)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
worker_17652569,70,[],0.00,0.00,0.00,0,0,0.0
worker_16498372,58,"[0, 1]",0.06,0.06,0.04,3,0,1.0
worker_18465660,80,"[0, 1]",0.06,0.06,0.06,3,0,1.0
worker_12572470,38,"[0, 1]",0.04,0.04,0.04,2,0,1.0
worker_18975312,88,"[0, 1]",0.04,0.04,0.02,2,0,1.0


So the crowd labels in general are quite good! But how much of our dev and training
sets do they cover?

In [9]:
print(f"Training set coverage: {100 * LFAnalysis(L_train).label_coverage(): 0.1f}%")
print(f"Dev set coverage: {100 * LFAnalysis(L_dev).label_coverage(): 0.1f}%")

Training set coverage:  50.3%
Dev set coverage:  50.0%


### Additional labeling functions

To improve coverage of the training set, we can mix the crowdworker labeling functions with labeling
functions of other types.
For example, we can use [TextBlob](https://textblob.readthedocs.io/en/dev/index.html), a tool that provides a pretrained sentiment analyzer. We run TextBlob on our tweets and create some simple LFs that threshold its polarity score, similar to what we did in the spam_tutorial.

In [10]:
from snorkel.labeling import labeling_function
from snorkel.preprocess import preprocessor
from textblob import TextBlob


@preprocessor(memoize=True)
def textblob_polarity(x):
    scores = TextBlob(x.tweet_text)
    x.polarity = scores.polarity
    return x


# Label high polarity tweets as positive.
@labeling_function(pre=[textblob_polarity])
def polarity_positive(x):
    return 1 if x.polarity > 0.3 else -1


# Label low polarity tweets as negative.
@labeling_function(pre=[textblob_polarity])
def polarity_negative(x):
    return 0 if x.polarity < -0.25 else -1


# Similar to polarity_negative, but with higher coverage and lower precision.
@labeling_function(pre=[textblob_polarity])
def polarity_negative_2(x):
    return 0 if x.polarity <= 0.3 else -1

### Applying labeling functions to the training set

In [11]:
text_lfs = [polarity_positive, polarity_negative, polarity_negative_2]
lfs = text_lfs + worker_lfs

applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
L_dev = applier.apply(df_dev)

  0%|          | 0/187 [00:00<?, ?it/s]

 18%|█▊        | 34/187 [00:00<00:00, 336.60it/s]

 46%|████▌     | 86/187 [00:00<00:00, 376.05it/s]

 74%|███████▍  | 139/187 [00:00<00:00, 410.40it/s]

100%|██████████| 187/187 [00:00<00:00, 469.41it/s]


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:00<00:00, 515.15it/s]

In [12]:
LFAnalysis(L_dev, lfs).lf_summary(Y_dev).head()

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
polarity_positive,0,[1],0.30,0.16,0.12,15,0,1.000000
polarity_negative,1,[0],0.10,0.10,0.04,5,0,1.000000
polarity_negative_2,2,[0],0.70,0.40,0.32,26,9,0.742857
worker_6332651,3,"[0, 1]",0.06,0.06,0.06,1,2,0.333333
worker_6336109,4,[],0.00,0.00,0.00,0,0,0.000000


Using the text-based LFs, we've expanded coverage on both our training set
and dev set to 100%.
We'll now take these noisy and conflicting labels, and use the LabelModel
to denoise and combine them.

In [13]:
print(f"Training set coverage: {100 * LFAnalysis(L_train).label_coverage(): 0.1f}%")
print(f"Dev set coverage: {100 * LFAnalysis(L_dev).label_coverage(): 0.1f}%")

Training set coverage:  100.0%
Dev set coverage:  100.0%


## Train LabelModel And Generate Probabilistic Labels

In [14]:
from snorkel.labeling import LabelModel

# Train LabelModel.
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=100, seed=123, log_freq=20, l2=0.1, lr=0.01)

As a spot-check for the quality of our LabelModel, we'll score it on the dev set.

In [15]:
from snorkel.analysis import metric_score

preds_dev = label_model.predict(L_dev)

acc = metric_score(Y_dev, preds_dev, probs=None, metric="accuracy")
print(f"LabelModel Accuracy: {acc:.3f}")

LabelModel Accuracy: 0.920


We see that we get very high accuracy on the development set.
This is due to the abundance of high quality crowdworker labels.
**Since we don't have these high quality crowdsourcing labels for the
test set or new incoming data points, we can't use the LabelModel reliably
at inference time.**
In order to run inference on new incoming data points, we need to train a
discriminative model over the tweets themselves.
Let's generate a set of labels for that training set.

In [16]:
preds_train = label_model.predict(L_train)

## Use Soft Labels to Train End Model

### Getting features from BERT
Since we have very limited training data, we cannot train a complex model like an LSTM with a lot of parameters.
Instead, we use a pre-trained model, [BERT](https://github.com/google-research/bert), to generate embeddings for each our tweets, and treat the embedding values as features.
This may take 5-10 minutes on a CPU, as the BERT model is very large.

In [17]:
import numpy as np
import torch
from pytorch_transformers import BertModel, BertTokenizer

model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


def encode_text(text):
    input_ids = torch.tensor([tokenizer.encode(text)])
    return model(input_ids)[0].mean(1)[0].detach().numpy()


X_train = np.array(list(df_train.tweet_text.apply(encode_text).values))
X_test = np.array(list(df_test.tweet_text.apply(encode_text).values))

### Model on labels
Now, we train a simple logistic regression model on the BERT features, using labels
obtained from our LabelModel.

In [18]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(solver="liblinear")
sklearn_model.fit(X_train, preds_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
print(f"Accuracy of trained model: {sklearn_model.score(X_test, Y_test)}")

Accuracy of trained model: 0.86


We now have a trained model that can be applied to future data points without requiring crowdsourced labels, and with accuracy not much lower than the `LabelModel` that _does_ have access to crowdsourced labels!

## Summary

In this tutorial, we accomplished the following:
* We demonstrated how to combine crowdsourced labels with other programmatic LFs to improve coverage.
* We used the `LabelModel` to combine inputs from crowdworkers and other LFs to generate high quality probabilistic labels.
* We used our labels to train a classifier for making predictions on new, unseen data points.